# Analyse et Visualisation des Données de Monitors

Ce Notebook a pour objectif d'analyser et de visualiser les données concernant les monitors collectées sur les plateformes **eBay** et **Flipkart**.

Les étapes réalisées incluent :

- **Importation des librairies** : Manipulation des données, création de graphiques, gestion des chemins et journalisation.
- **Configuration des chemins** : Définition des répertoires du projet pour les données nettoyées et les résultats.
- **Chargement des données nettoyées** : Lecture et préparation des fichiers CSV pour la catégorie *monitors*.
- **Filtrage des produits cross-plateformes** : Identification des monitors ayant des spécifications identiques sur les deux plateformes.
- **Analyse et visualisation des différences de prix** : Agrégation des données et génération de graphiques pour observer les tendances de prix.

Chaque fonction est expliquée en détail ci-dessous.


# Importation des Librairies et Configuration du Logging

Dans cette section, nous importons les bibliothèques nécessaires pour :
- La manipulation de données avec **pandas**.
- La création de graphiques avec **matplotlib** et **seaborn**.
- La gestion des chemins avec **pathlib**.
- Le traitement des chaînes de caractères avec **re**.
- La journalisation des opérations avec **logging**.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Configuration des Chemins du Projet

Ici, nous définissons les chemins essentiels du projet :
- `PROJECT_ROOT` : La racine du projet.
- `CLEANED_DATA_PATH` : Le dossier contenant les données nettoyées.
- `RESULTS_PATH` : Le dossier où seront sauvegardés les résultats (graphiques générés).


In [ ]:
# Configure paths
PROJECT_ROOT = Path(__file__).resolve().parents[2]
CLEANED_DATA_PATH = PROJECT_ROOT / 'data' / 'cleaned'
RESULTS_PATH = PROJECT_ROOT / 'results'

# Chargement des Données Nettoyées pour les Monitors

La fonction `load_cleaned_data` charge les fichiers CSV nettoyés pour la catégorie **monitors** provenant des plateformes *eBay* et *Flipkart*.  
Pour chaque fichier, elle effectue les opérations suivantes :
- **Standardisation des noms de colonnes** : Conversion en minuscules et remplacement des espaces par des underscores.
- **Validation des colonnes requises** : Vérification de la présence des colonnes essentielles (par exemple, *title*, *price*, *screen_size_in*, etc.).
- **Ajout de l’identifiant de plateforme**.
- **Normalisation des champs** : Application des fonctions `normalize_brand` et `normalize_aspect_ratio` ainsi que la conversion de la taille de l’écran en nombre entier.
- **Traitement de la date de collecte** : Conversion de la colonne `collection_date` en date.
  
Les DataFrames obtenus sont ensuite concaténés pour former un DataFrame global.


In [ ]:
def load_cleaned_data():
    """Load cleaned data for monitors from eBay and Flipkart"""
    platforms = ['ebay', 'flipkart']
    dfs = []
    
    required_columns = {
        'title', 'price', 'screen_size_in', 'aspect_ratio', 
        'refresh_rate_hz', 'response_time_ms', 'brand', 'collection_date'
    }
    
    for platform in platforms:
        data_dir = CLEANED_DATA_PATH / platform / 'monitors'
        if not data_dir.exists():
            logger.warning(f"Missing data directory: {data_dir}")
            continue
            
        for file in data_dir.glob('*.csv'):
            try:
                df = pd.read_csv(file)
                df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
                
                # Validate columns
                missing_cols = required_columns - set(df.columns)
                if missing_cols:
                    logger.warning(f"Missing columns in {file}: {missing_cols}")
                    continue
                
                # Add platform identifier
                df['platform'] = platform
                
                # Normalize key fields
                df['brand'] = df['brand'].apply(normalize_brand)
                df['aspect_ratio'] = df['aspect_ratio'].apply(normalize_aspect_ratio)
                df['screen_size_in'] = df['screen_size_in'].apply(
                    lambda x: round(float(x)) if pd.notnull(x) else None
                )
                
                # Handle collection date
                df['collection_date'] = pd.to_datetime(
                    df['collection_date'], errors='coerce'
                ).dt.date
                
                dfs.append(df)
                
            except Exception as e:
                logger.error(f"Error loading {file}: {str(e)}")
                continue
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


# Filtrage des Produits Cross-plateformes

La fonction `filter_products_by_platforms` identifie les monitors présentant des spécifications identiques sur les deux plateformes.  
Pour cela, elle :
- Crée un identifiant unique (`product_id`) pour chaque produit en combinant plusieurs champs (brand, screen size, aspect ratio, refresh rate, response time).
- Regroupe les produits par `product_id` et sélectionne ceux présents sur **eBay** et **Flipkart**.
  
Le DataFrame retourné contient uniquement les monitors disponibles sur les deux plateformes.


In [ ]:
def filter_products_by_platforms(df: pd.DataFrame):
    """Find monitors with matching specifications across platforms"""
    # Create a unique product identifier
    df['product_id'] = (
        df['brand'] + "|" + 
        df['screen_size_in'].astype(str) + "|" + 
        df['aspect_ratio'].astype(str) + "|" + 
        df['refresh_rate_hz'].astype(str) + "|" + 
        df['response_time_ms'].astype(str)
    )
    
    # Find products present on both platforms
    platform_groups = df.groupby('product_id')['platform'].unique()
    cross_platform = platform_groups[platform_groups.apply(
        lambda x: len(set(x) & {'ebay', 'flipkart'}) >= 2
    )].index.tolist()
    
    return df[df['product_id'].isin(cross_platform)]

# Analyse et Visualisation des Différences de Prix

La fonction `analyze_price_differences` analyse les différences de prix pour les monitors identifiés sur les deux plateformes.  
Les étapes réalisées comprennent :
- **Agrégation** : Calcul du prix moyen par date de collecte pour chaque monitor et par plateforme.
- **Visualisation** : Création d’un graphique en barres pour chaque produit, affichant l’évolution des prix au fil du temps.
- **Sauvegarde** : Chaque graphique est enregistré dans le dossier `RESULTS_PATH / 'monitors'` avec un nom de fichier dérivé de l’identifiant du produit.


In [ ]:
def analyze_price_differences(filtered_df: pd.DataFrame):
    """Analyze and visualize price differences for matched monitors"""
    if filtered_df.empty:
        logger.info("No cross-platform monitors found for analysis")
        return
    
    # Group by product_id and platform
    grouped = filtered_df.groupby(['product_id', 'platform', 'collection_date'])['price'].mean().unstack(level='platform')
    
    # Generate bar plots for each monitor
    for product_id, data in grouped.groupby(level=0):
        plt.figure(figsize=(12, 6))
        
        # Plot bar graph
        data.plot(kind='bar', figsize=(12, 6))
        
        # Set title and labels
        plt.title(f"Price Trends for {product_id.replace('|', ' ')}")
        plt.xlabel("Collection Date")
        plt.ylabel("Price (USD)")
        
        # Format x-axis dates
        plt.xticks(range(len(data.index)), data.index.get_level_values('collection_date'), rotation=45, ha='right')
        
        # Add legend and grid
        plt.legend(title='Platform')
        plt.grid(True)
        plt.tight_layout()
        
        # Save the plot
        sanitized_title = re.sub(r"[^\w\s]", "_", product_id.replace("|", "_"))
        file_name = f"Monitor_{sanitized_title}.png"
        # Assurez-vous que le dossier de destination existe
        (RESULTS_PATH / 'monitors').mkdir(exist_ok=True, parents=True)
        plt.savefig(RESULTS_PATH / 'monitors' / file_name)
        plt.close()


# Exécution Principale

La section suivante orchestre l'exécution complète du processus :
1. **Création des dossiers de résultats** s'ils n'existent pas.
2. **Chargement et traitement des données** via `load_cleaned_data()`.
3. **Filtrage des produits cross-plateformes** avec `filter_products_by_platforms()`.
4. **Analyse et visualisation des différences de prix** à l'aide de `analyze_price_differences()`.

En cas d'erreur, un message détaillé est loggé.


In [ ]:
if __name__ == "__main__":
    # Créer les dossiers de résultats s'ils n'existent pas
    RESULTS_PATH.mkdir(exist_ok=True)
    (RESULTS_PATH / 'monitors').mkdir(exist_ok=True)
    
    try:
        logger.info("Loading and processing monitor data...")
        df = load_cleaned_data()
        
        if df.empty:
            logger.error("No cleaned data found. Check data/cleaned directories.")
            exit(1)
            
        logger.info(f"Loaded {len(df)} records from cleaned data")
        
        filtered_df = filter_products_by_platforms(df)
        logger.info(f"Found {len(filtered_df)} cross-platform monitor entries")
        
        analyze_price_differences(filtered_df)
        
    except Exception as e:
        logger.error(f"Critical error: {str(e)}", exc_info=True)
